In [4]:
import os
import cv2
import shutil
import deeplabcut

Loading DLC 2.3.1...


In [3]:
! gdown --id 1kipiRhpnCkbYW974jDSD08xKJ2pBAONP


/usr/local/lib/python3.9/dist-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1kipiRhpnCkbYW974jDSD08xKJ2pBAONP
To: /content/Ken_Test_Original.mp4
100% 12.4M/12.4M [00:00<00:00, 17.3MB/s]


In [ ]:
! gdown --id 17quQdW2yB5pDX_WNVLi599Ek_gQFuHpa

In [ ]:
!pip install deeplabcut[gui,tf]

In [12]:


file = '/content/output2_gcorr.mp4'
file3 = '/content/output4.mp4'
file2 = os.path.join(
    os.path.abspath(os.getcwd()),
    os.path.basename(file),
)
bodyparts = ['RightEye_Pupil', 'LeftEye_Pupil',
             'NostrilsTop_Centre', 'OutlineTop_Mid']
videotype = os.path.splitext(file)[-1].lstrip('.')  # or MOV, or avi, whatever you uploaded!
video_down = file2

model_options = deeplabcut.create_project.modelzoo.Modeloptions
model_selection = 'primate_face'



In [14]:
# import subprocess

# input_file = '/content/output2_gcorr.mp4'
# output_file = '/content/output4.mp4'
# video_codec = 'libx264'
# audio_codec = 'aac'

# cmd = ['ffmpeg', '-i', input_file, '-c:v', video_codec, '-c:a', audio_codec, output_file]
# subprocess.run(cmd)

In [ ]:
! gdown --id 10k_aIV-dBc1OLcqGJ3o4ZMu2lD9qQtmi

In [13]:
project_name = 'DLC_GazeXBI'
your_name = 'anc'

config_path, train_config_path = deeplabcut.create_pretrained_project(
    project_name,
    your_name,
    [video_down],
    videotype=videotype,
    model=model_selection,
    analyzevideo=True,
    createlabeledvideo=False,
    copy_videos=True,
)



Created "/content/DLC_GazeXBI-anc-2023-03-13/videos"
Created "/content/DLC_GazeXBI-anc-2023-03-13/labeled-data"
Created "/content/DLC_GazeXBI-anc-2023-03-13/training-datasets"
Created "/content/DLC_GazeXBI-anc-2023-03-13/dlc-models"
Copying the videos
/content/DLC_GazeXBI-anc-2023-03-13/videos/output2_gcorr.mp4


FileNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
edits = {
    'dotsize': 3,  # size of the dots!
    'pcutoff': 0.1,  # the higher, the more conservative the plotting!
}
deeplabcut.auxiliaryfunctions.edit_config(config_path, edits)

project_path = os.path.dirname(config_path)
full_video_path = os.path.join(
    project_path,
    'videos',
    os.path.basename(video_down),
)

# filter predictions (should already be done above ;):
deeplabcut.filterpredictions(config_path, [full_video_path], videotype=videotype)

# re-create the video with your edits!
deeplabcut.create_labeled_video(config_path, [full_video_path], videotype=videotype, displayedbodyparts=bodyparts,
                                filtered=True)


Filtering with median model C:\Users\baner\OneDrive\Desktop\lab rotation 2\DLC_GazeXBI-anc-2023-03-09\videos\output2_gcorr.mp4
Saving filtered csv poses!


In [7]:
#gamma correction
import cv2
import numpy as np

# Open the video file
cap = cv2.VideoCapture('/content/output3.mp4')

# Define the gamma value
gamma = 2

# Set the video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output2_gcorr.mp4', fourcc, fps, (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))


# Read the video frames and perform gamma correction
while cap.isOpened():
    # Read a frame from the video
    ret, frame = cap.read()

    # If the frame was not read successfully, exit the loop
    if not ret:
        break

    # Normalize the pixel values to the range [0, 1]
    frame_normalized = frame.astype(np.float32) / 255.0

    # Apply gamma correction
    frame_corrected = np.power(frame_normalized, 1/gamma)

    # Scale the pixel values back to the range [0, 255]
    frame_scaled = (frame_corrected * 255.0).astype(np.uint8)

    out.write(frame_scaled)
    
# Release the video file and close the window
cap.release()
cv2.destroyAllWindows()


In [6]:
#cropping video
# Load the video file
video = cv2.VideoCapture('/content/Ken_Test_Original.mp4')

# Get the frame rate and total number of frames
fps = int(video.get(cv2.CAP_PROP_FPS))
total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

# Set the start and end points to crop out
start_sec = 240  # Crop out the first 5 seconds
end_sec = 120 # Crop out the last 10 seconds
start_frame = int(start_sec * fps)
end_frame = int((total_frames / fps) - end_sec) * fps

# Set the video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output3.mp4', fourcc, fps, (int(video.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))))

# Set the current frame number to the start frame
video.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

# Loop through the frames and write them to the output video
for i in range(start_frame, end_frame):
    ret, frame = video.read()
    if not ret:
        break
    out.write(frame)

# Release the video objects
video.release()
out.release()
cv2.destroyAllWindows()